# Jetson Autopilot - Data Collection

This notebook guides you through collecting training data for the autopilot system.

In [ ]:
import sys
sys.path.insert(0, '..')

from pathlib import Path
from IPython.display import display
import ipywidgets as widgets
import time

## Configuration

In [ ]:
from jetson import Config, DataCollector

# Configure paths and settings
config = Config()
config.datasets_dir = Path("/home/jetson/datasets")
config.car.steering_offset = 0.035
config.car.throttle_gain = 0.7

# Dataset naming
DATASET_NAME = "session1"
DATASET_MODE = "training"  # "training", "validation", or "testing"

print(f"Output directory: {config.datasets_dir / f'{DATASET_NAME}_{DATASET_MODE}'}")

## Initialize Hardware

In [ ]:
collector = DataCollector(
    config=config,
    dataset_name=DATASET_NAME,
    dataset_mode=DATASET_MODE,
)
collector.setup()

print("Hardware initialized successfully")

## Setup Gamepad Controls

Button mappings (Gamepad Mode 2):
- Left stick: Throttle
- Right stick: Steering  
- Button X: Start recording
- Button B: Save recording
- Button RB: Half throttle
- Button RT: Brake

In [ ]:
from jetson.data_collection import setup_gamepad_controls

controller = setup_gamepad_controls(collector)
display(controller)
time.sleep(1.0)

print("Gamepad connected. Use buttons to control recording.")

## Camera Preview (Optional)

In [ ]:
import traitlets
import cv2

SHOW_PREVIEW = False  # Set to True to show camera preview

if SHOW_PREVIEW:
    image_widget = widgets.Image(format='jpeg', width=448, height=336)
    display(image_widget)
    
    def update_preview(frame):
        collector.capture_frame()
        _, encoded = cv2.imencode('.jpg', frame)
        return bytes(encoded)
    
    camera_link = traitlets.dlink(
        (collector.camera, 'value'),
        (image_widget, 'value'),
        transform=update_preview
    )
else:
    # Still capture frames without preview
    def capture_only(frame):
        collector.capture_frame()
        return frame
    
    collector.camera.observe(
        lambda change: capture_only(change['new']),
        names='value'
    )

## Manual Controls

You can also control recording programmatically:

In [ ]:
# Start a new recording session
# collector.start_recording()

In [ ]:
# Save the current recording
# collector.save_recording()

In [ ]:
# Discard the current recording
# collector.discard_recording()

## Tips

1. Collect at least 20,000 samples for good results (60,000+ recommended)
2. Include variety: different speeds, turns, lighting conditions
3. Record stopping scenarios with obstacles/cones
4. Record crossroad turns if needed
5. Split data: 70% training, 20% validation, 10% testing